In [4]:
import numpy as np
import pandas as pd
import seaborn as sb
% matplotlib inline
from datetime import datetime, timedelta

from bs4 import BeautifulSoup as bs
import requests
import re
import pickle

In [37]:
rankings_p = '../data/rankings.p'
headers_p = '../data/headers.p'

# function to scrape marathon data
def scrape_marathon_data():
    years = range(2002, 2018)
    letters = [chr(x + ord('A')) for x in range(0, 26)]

    URL = "https://services.datasport.com/{}/lauf/lamara/ALFA{}.HTM"
    
    rankings = {}
    headers = {}
    for year in years:
        print(str(year) + ": ", end='')
        rankings[str(year)] = []
        for letter in letters:
            r = requests.get(URL.format(year, letter))
            soup = bs(r.text, 'html.parser')
            title = soup.find('title').getText()
            if(title != 'Adresse nicht vorhanden / The address is not available'):
                data = soup.findAll('font', {'size':'2'})

                print(letter, end='')
                headers[str(year)] = data[0].getText()
                rankings[str(year)].extend(re.compile('¦ *\r\n').split(data[1].getText()))
            else:
                print("Skipped")
        print('')

    # Pickle data
    pickle.dump(rankings, open(rankings_p, 'wb'))
    pickle.dump(headers, open(headers_p, 'wb'))
    return rankings, headers

# Only scrape if pickle unavailable
def get_data(force_scrape=False):
    if force_scrape:
        print('Scrape data from website')
        return scrape_marathon_data()
    try:
        print('Trying to load data from pickle')
        return pickle.load(open(rankings_p, 'rb')), pickle.load(open(headers_p, 'rb'))
    except (OSError, IOError) as e:
        print('Failed to load pickle:')
        print(e)
        print('Scrape data from website')
        return scrape_marathon_data()


Trying to load data from pickle


In [106]:
"""
This function finds the position of where the pages should be split into columns
based on whitespace
"""
def parse_columns(ranking, header):
    column_structure = {}
    column_mask = np.ones(np.array(list(ranking[0])).shape, dtype=bool)
    spaces_mask = np.ones(np.array(list(ranking[0])).shape)*32
    
    # create mask of space positions for field delimiter detection
    for line in ranking[:-1]:
        cur_line = np.array(list(line)).view(np.uint32)
        
        try:
            column_mask = np.logical_and(column_mask, np.equal(cur_line, spaces_mask))
        except:
            #print("Skipped line:", line)
            #print(cur_line.shape, column_mask.shape)
            pass
        
    # include header in space position mask
    header_int = np.array(list(header[:column_mask.shape[0]])).view(np.uint32)
    column_mask = np.logical_and(column_mask, np.equal(header_int, spaces_mask))
        
    # find field delimiter positions
    inside_space_col = False
    delimiters = []
    for i, is_space_col in enumerate(list(column_mask)):
        if is_space_col:
            if not inside_space_col:
                inside_space_col = True
                delimiters.append(i)
                
        elif inside_space_col:
            inside_space_col = False
            
    return delimiters
    

"""
Seperates the lists of lines into columns and create a dictionary of dataframes
"""
def separate_cols(rankings, headers):
    # dictionary to keep dataframes in
    dfs = {}
    for year in rankings:
        ranking = rankings[year]
        header = headers[year]
        
        # fix offset headers in 2007-2009 data
        if year in ['2007', '2008', '2009']:
            header = header.replace('an lieu', 'an   lieu')
        delimiters = parse_columns(ranking, header)

        # find positions for each field
        start = 0
        cols = {} 
        unnamed = 0
        for delimiter in delimiters:
            key = header[start:delimiter+1].strip()
            if key == '':
                key = unnamed
                unnamed += 1
            cols[key] = (start, delimiter+1)
            start = delimiter+1

        start = 0
        result = {}
        for i, line in enumerate(ranking):
            if '\r\ntotal ' not in line and '\r\nTotal' not in line:
                for col in cols:
                    start, end = cols[col]
                    if col in result:
                        result[col].append(line[start:end].strip())
                    else:
                        result[col] = [line[start:end].strip()]
        dfs[year] = pd.DataFrame(result, columns=result.keys()) 
    print('Columns separated')
    return dfs

In [41]:
mapping = {
    'lieu': '',
    0: 'DELETE',
    2: 'DELETE',
    3: 'DELETE',
    'catégorie': '',
    'nom': '',
    1: 'DELETE',
    'temps': '',
    'Jg': 'an',
    'an lieu': '',
    'overall': '',
    'retard': '',
    'équipe': '',
    'moyenne': '',
    'Rang': 'rang',
    'nom/lieu': '',
    'Stnr': 'doss',
    'an': '',
    'pénalité': '',
    'doss': '',
    'temps net': '',
    'rang': '',
    '¦': 'DELETE',
    'équipe/lieu': '',
    'Rückstand': 'retard',
    'Kategorie': 'catégorie',
    'Team/Ortschaft': 'équipe/lieu',
    'Name/Ort': 'nom/lieu',
    'Zeit': 'temps'
}

In [105]:
"""
Where possible, this function matches the names of the columns 
of the individual dataframes
"""
def normalize_column_names(dfs_):
    dfs_c = dfs_.copy()
    for year, _ in dfs_c.items():
        for header_name in list(dfs_c[year]):
            if mapping[header_name] == 'DELETE':
                dfs_c[year] = dfs_c[year].drop(header_name, axis=1)
            elif mapping[header_name] != "":
                dfs_c[year] = dfs_c[year].rename(columns = {header_name : mapping[header_name]})
                print("Renamed {} to {} in year {}".format(
                    header_name, mapping[header_name], year))
    print('Column names normalized')
    return dfs_c
                


def print_col_overview(dfs):
    # fields present in all years
    shared_fields = set.intersection(*[set(v) for k, v in dfs.items()])
    print("Shared fields:", shared_fields)

    fields = set()
    for year in sorted(dfs):
        fields = fields.union(set(list(dfs[year])))
        print(year)
        print(set(dfs[year]).difference(shared_fields))
    print("Union remaining fields:", fields.difference(shared_fields))

In [104]:
def fix_cities(dfs_):
    dfs_c = dfs_.copy()
    """
    Years 2002-2006 have a team/city field and a name/city field.
    If a team is provided, the city is added to the end of the name,
    after a comma (',').
    
    This function removes the teams and converts the column 'équipe/lieu' 
    to a dedicated 'lieu' column
    """
    for year, df in dfs_c.items():
        col_name = 'nom/lieu'
        col_team = 'équipe/lieu'
        if col_name in df:
            for i, row in df.iterrows():
                m = re.compile('(.*), (.*)').match(row[col_name])
                if m is not None:
                    dfs_c[year].at[i, col_name] = m.group(1)
                    dfs_c[year].at[i, col_team] = m.group(2)
            # rename columns
        rename_dict = {col_name : 'nom', col_team: 'lieu'}
        dfs_c[year].rename(columns=rename_dict, inplace=True)
    print('Cities fixed')
    return dfs_c

In [103]:
"""
Returns the fields that are present in all years
"""
def get_shared_fields(dfs):
    return set.intersection(*[set(v) for k, v in dfs.items()])

"""
Returns the fields that are only present in some of the years
"""
def get_unshared_fields(dfs):
    return set.union(*[set(v) for k, v in dfs.items()]).difference(get_shared_fields(dfs))

"""
Deletes columns from dataframes that are only present in some of the years
"""
def drop_uncommon_fields(dfs_):
    dfs_c = dfs_.copy()
    for year, df in dfs_c.items():
        dfs_c[year] = dfs_c[year][list(get_shared_fields(dfs_c))]
    print('Uncommon fields dropped')
    return dfs_c

In [107]:
def create_year_column(dfs_):
    dfs_c = dfs_.copy()
    for year in dfs_c:
        df = dfs_c[year].copy()
        df['race-year'] = int(year)
        dfs_c[year] = df
    print('Year column created')
    return dfs_c

In [80]:
def create_merged_df(dfs_):
    return pd.concat(list(dfs_.values()))

In [63]:
def normalize_categories(df_):
    """
    rename important categories and drop unused
    """
    df_c = df_.copy()
    df_c['catégorie'] = df_['catégorie'].replace({
        r'^(\d\d)[-/][MQS]*([DH])\d+$' : r'\1|\2',
        r'^Q([DH])\d$' : r'10|\1',
        r'^S([DH])\d$' : r'21|\1',
        r'^M([DH])\d$' : r'42|\1'
    }, regex = True)
    
    # columns to keep
    keep = [str(x) + '|H' for x in [10, 21, 42]]
    keep.extend([str(x) + '|D' for x in [10, 21, 42]])
    
    # drop all other rows
    df_c = df_c[df_c['catégorie'].isin(keep)]
    
    # split catégorie into two columns
    df_c[['distance', 'gender']] = df_c['catégorie'].str.split(
        '|', expand=True
    )
    
    # replace german gender abbreviations
    df_c['gender'] = df_c['gender'].replace({
        'H' : 'male',
        'D' : 'female'
    })
    
    df_c = df_c.drop('catégorie', axis=1)
    
    return df_c
    

In [108]:
# Get the data
rankings, headers = get_data()
# Separate data into columns and build dataframes
dfs = separate_cols(rankings, headers)
# Make sure the same columns have the same column names
dfs_n = normalize_column_names(dfs)
# Fix cities which are in the name column in some cases
dfs_f = fix_cities(dfs_n)
# Drop columns that are unique to some years
dfs_d = drop_uncommon_fields(dfs_f)
# Create year columns
dfs_y = create_year_column(dfs_d)

Trying to load data from pickle
Columns separated
Renamed Stnr to doss in year 2002
Renamed Kategorie to catégorie in year 2002
Renamed Name/Ort to nom/lieu in year 2002
Renamed Zeit to temps in year 2002
Renamed Rang to rang in year 2002
Renamed Jg to an in year 2002
Renamed Team/Ortschaft to équipe/lieu in year 2002
Renamed Rückstand to retard in year 2002
Column names normalized
Cities fixed
Uncommon fields dropped
Year column created


In [109]:
# Create final dataframe
df = create_merged_df(dfs_y)

In [112]:
# Print overview of columns for inspection
df.head()

an   retard                 nom     doss                  lieu  \
0  1981  45.43,3  Aarskog Kay-Morten  (16798)              N-Oteren   
1  1994  53.40,3         Abadie Théo   (3616)           Ecublens VD   
2  1966  23.51,0      Abaidia Jilani   (4078)  St-Légier-La Chiésaz   
3  1972  30.40,6    Abaidia Sandrine   (4076)             St-Légier   
4  1991  23.59,5        Abawi Khaled  (12280)             Renens VD   

       temps  rang  race-year distance  gender  
0  1:15.48,7  676.       2017       10    male  
1  2:03.37,1  484.       2017       21    male  
2  1:45.25,0  139.       2017       21    male  
3  1:53.23,1   77.       2017       21  female  
4    53.23,8  428.       2017       10    male

In [113]:
# Save dataframe to csv
df.to_csv('../data/marathon-data.csv', sep='\t')